# A Simple Bitcoin Demo in Python

## 1 Merkle Tree

In [1]:
import hashlib

def double_sha256(byts):
    '''Purpose SHA256 twice to the bytes.'''
    return hashlib.sha256(hashlib.sha256(byts).digest()).digest()

In [2]:
class MerkleTree:
    def __init__(self):
        self.txid_list = []
        self.merkle_root = ''

    def make_merkle_tree(self, txid_list, is_sorted=True):
        '''Compute merkle root for the given list of TXIDs.'''
        if len(txid_list) == 0:
            raise ValueError("empty list")

        # Sort TXIDs (strings) in lexicographical order if not ordered
        if is_sorted:
            self.txid_list = txid_list
        else:
            self.txid_list = sorted(txid_list)

        # Compute merkle root row by row
        hashes = [s.encode() for s in self.txid_list]
        while len(hashes) > 1:
            if len(hashes) & 1:  # Number of hashes is odd
                hashes.append(hashes[-1])
            next_hashes = []
            for i in range(0, len(hashes), 2):
                next_hashes.append(hashes[i] + hashes[i+1])
            hashes = [double_sha256(s) for s in next_hashes]  # Hashes are bytes, not texts

        self.merkle_root = hashes[0]
        return self.merkle_root  # 32 bytes

## 2 Bitcoin Accounts

In [3]:
import ecdsa
from ecdsa import SigningKey, VerifyingKey, SECP256k1
import base58

class Account:
    pubkey_type = b'\x04'  # The identification byte 0x04 denotes uncompressed coordinates of public key
    
    def __init__(self, prikey=''):
        '''
        Create a Bitcoin account using the given private key.
        If private key not provided, generate a new one randomly.
        '''
        if prikey:
            self.private_key = prikey
            self.public_key = self.gen_pubkey_from_prikey(prikey)
        else:
            self.private_key, self.public_key = self.gen_keypair()
        
        self.public_key_hash = self.gen_public_key_hash(self.public_key)
        self.address = self.gen_address_from_pkh(self.public_key_hash)

    @staticmethod
    def gen_keypair():
        '''Generate a new pair of keys using SECP256k1 ECDSA.'''
        sk = SigningKey.generate(curve=SECP256k1) # Private key
        vk = sk.verifying_key                     # Public key (without prefix)
        return sk.to_string(), Account.pubkey_type + vk.to_string()
        
    @staticmethod
    def gen_pubkey_from_prikey(prikey):
        '''Compute public key from the given private key.'''
        sk = SigningKey.from_string(prikey, curve=SECP256k1)
        vk = sk.verifying_key
        return Account.pubkey_type + vk.to_string()

    @staticmethod
    def gen_public_key_hash(pubkey):
        '''Compute public key hash with "double hash".'''
        # Note that pubkey is already prefixed
        temp = hashlib.sha256(pubkey).digest()
        h = hashlib.new('ripemd160')
        h.update(temp)
        return h.digest()

    @staticmethod
    def gen_address_from_pkh(pubkeyhash):
        '''Compute base58check encoded address.'''
        # Base58check encode
        version_prefix = b'\x00'  # Version prefix of Bitcoin addresses is 0x00
        return base58.b58encode_check(version_prefix + pubkeyhash)

In [4]:
# Example from textbook
example_account = Account(bytes.fromhex("1E99423A4ED27608A15A2616A2B0E9E52CED330AC530EDCC32C8FFC6A526AEDD"))
print("Public key:", example_account.public_key.hex())
print("Address", example_account.address)

Public key: 04f028892bad7ed57d2fb57bf33081d5cfcf6f9ed3d3d7f159c2e2fff579dc341a07cf33da18bd734c600b96a72bbc4749d5141c90ec8ac328ae52ddfe2e505bdb
Address b'1424C2F4bC9JidNjjTUZCbUxv6Sa1Mt62x'


In [5]:
# Generate 100 Bitcoin accounts for testing
test_accounts = [Account() for i in range(100)]

## 3 Signature and Verification

In [6]:
test_message = b"blockchain-ss-2021"
print(test_message.hex())

626c6f636b636861696e2d73732d32303231


In [7]:
def sign(message, prikey):
    '''Sign a byte message using the private key.'''
    sk = SigningKey.from_string(prikey, curve=SECP256k1)
    return sk.sign(message)
    
def verify(message, signature, pubkey):
    '''
    Verify signature using the public key.
    Returns true if succeed. Otherwise raises errors.
    '''
    vk = VerifyingKey.from_string(pubkey, curve=SECP256k1)
    return vk.verify(signature, message)

In [8]:
fail_flag = False

for account in test_accounts:
    # Sign the message
    test_signature = sign(test_message, account.private_key)
    
    # Verify the signature
    try:
        verify(test_message, test_signature, account.public_key[1:])
    except ecdsa.BadSignatureError as e:
        print(e)
        fail_flag = True
        
if not fail_flag:
    print("All succeeded.")

All succeeded.


In [9]:
# A bad case
test_signature = sign(test_message, test_accounts[0].private_key)
temp = list(test_signature)
temp[-1] += 1  # Modifying one byte in the signature causes corruption
bad_signature = bytes(temp)

print("Correct signature:", test_signature.hex())
print("Corrupt signature:", bad_signature.hex())

try:
    verify(test_message, bad_signature, test_accounts[0].public_key)
except ecdsa.BadSignatureError as e:
    print("BadSignatureError:", e)

Correct signature: b3869321ee65b2eec5af4bc7a8dddda57b439668e621ad6bda129769af3f81cdf4d6e99f61d31c905d2124e84d3a978963fa84db8dba0433b68e0287a783583d
Corrupt signature: b3869321ee65b2eec5af4bc7a8dddda57b439668e621ad6bda129769af3f81cdf4d6e99f61d31c905d2124e84d3a978963fa84db8dba0433b68e0287a783583e
BadSignatureError: Signature verification failed


## 4 Bitcoin Transactions

## 5 Bitcoin Blocks